In [1]:
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import MobileNetV2
from keras.optimizers import Adam
from keras.layers import AveragePooling2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Input
from keras.models import Model

In [2]:
BATCH_SIZE = 32
EPOCHS = 30
learning_rate = 1e-4
IMG_SIZE = (224, 224)

In [5]:
data = np.load('./processed-data/data.npy')
labels = np.load('./processed-data/labels.npy')

In [7]:
(train_data, test_data, train_label, test_label) = train_test_split(data, labels, test_size=0.1)

(train_data, validation_data, train_label, validation_label) = train_test_split(train_data, train_label, test_size=0.2)

In [8]:
aug = ImageDataGenerator(
	rotation_range=20,
	zoom_range=0.15,
	width_shift_range=0.2,
	height_shift_range=0.2,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest")

In [9]:
IMG_SHAPE = IMG_SIZE + (3, )

base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=IMG_SHAPE)

base_model.trainable = False

In [10]:
head_model = base_model.output
head_model = AveragePooling2D(pool_size=(7, 7))(head_model)
head_model = Flatten(name="flatten")(head_model)
head_model = Dense(128, activation="relu")(head_model)
head_model = Dropout(0.5)(head_model)
head_model = Dense(2, activation="softmax")(head_model)

In [11]:
model = Model(inputs=base_model.input, outputs=head_model)

In [12]:
model.compile(optimizer=Adam(lr=learning_rate, decay=learning_rate / EPOCHS), loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
fitted_model = model.fit(
                        aug.flow(train_data, train_label, batch_size=BATCH_SIZE),
                        steps_per_epoch=len(train_data) // BATCH_SIZE,
                        validation_data=(validation_data, validation_label),
                        epochs=EPOCHS)